In [1]:
import stackprinter  # type: ignore
import jupyter_black  # type: ignore
from dotenv import load_dotenv  # type: ignore
import time

from baml_agents import init_logging, with_model
from baml_client.async_client import b

init_logging()
stackprinter.set_excepthook()
load_dotenv()
jupyter_black.load()

b = with_model(b, "gpt-4.1-nano")

# Using BAML in Jupyter notebokos

## 1. Streaming outputs

BAML offers a great developer experience, and I used some utilities to recreate some of the magic in Jupyter notebooks.

In [2]:
from baml_agents import JupyterOutputBox

with JupyterOutputBox(clear_after_finish=False) as output_box:
    s = ""
    for c in "abcdefghijklmnopqrstuvwxyz" * 3:
        s += c
        output_box.update(s)
        time.sleep(0.008)

# The letters are generated and displayed as they are being generated.

abcdefghijklmnopqrstuvwxyzabcdefghijklmnopqrstuvwxyzabcdefghijklmnopqrstuvwxyz

In [3]:
from baml_agents import JupyterBamlCollector

with JupyterOutputBox(clear_after_finish=False) as s:
    c = JupyterBamlCollector(b, stream_callback=s.display(formatter="json"))
    await c.b.InteractiveBamlJupyter_WriteShortStory()

{
 "genre": "Fantasy",
 "characters": [
 {
 "name": "Lina",
 "age": 17,
 "occupation": "Apprentice Mage"
 },
 {
 "name": "Eldon",
 "age": 45,
 "occupation": "Ancient Wizard"
 }
 ],
 "story_summary": "Lina discovers Eldon's hidden library beneath the old forest, unlocking magic to save her village from a looming darkness, forging her destiny as a true mage."
}

![Image](https://github.com/user-attachments/assets/ca21b62a-48b7-4a16-99fa-20bbe61f68b2)

## 2. Inspecting prompts and completions

In [7]:
await c.display_calls()

![Image](https://github.com/user-attachments/assets/59ed3cf3-84df-45a3-bf18-44dfbc457e4e)

You can also group prompts and completions by LLM call and by LLM call session:

In [10]:
# Generate some calls
c = JupyterBamlCollector(b)
await c.b.InteractiveBamlJupyter_WriteShortStory()
await c.b.InteractiveBamlJupyter_WriteShortStory()

# Display the calls as a single session
await c.display_session("Story generation session")

![Image](https://github.com/user-attachments/assets/7bed8f67-efe9-48c7-918a-5d0853dded95)

## 3. Convenience class

The `JupyterOutputBox` and `JupyterBamlCollector` are available as a single convenience class:

In [14]:
from baml_agents import JupyterBamlMonitor

with JupyterBamlMonitor(b) as m:
    # Output is displayed in real time in the notebook
    await m.b.InteractiveBamlJupyter_WriteShortStory()
    # Outout is hidden from the notebook

# Buttons with prompt and completino are displayed in the notebook
await m.display_calls()

{'genre': 'Fantasy',
 'characters': [{'name': 'Luna', 'age': 17, 'occupation': 'Wizard-in-training'},
 {'name': 'Thorn', 'age': 25, 'occupation': 'Forest Guardian'},
 {'name': 'Eira', 'age': 30, 'occupation': 'Ancient Sorceress'}],
 'story_summary': "Luna seeks Eira's wisdom to unlock her powers, crossing woods guarded by Thorn, who distrusts outsiders, but fate binds them in a quest to save their realm from darkness."}

<IPython.core.display.Javascript object>

## 4. Interactive chat

In [ ]:
from baml_agents._jupyter_baml._chat_message import ChatMessage


async def callback(chat_history):
    msg = ChatMessage(
        content=f"I'm seeing {len(chat_history)} messages",
        role="assistant",
    )
    return [msg]


JupyterChatWidget(callback).display()